# Next Word Prediction Using LSTM

# Importing Required Libraries

In [134]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import Dropout
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical

# Text Data

In [135]:
data="Born and raised in New Delhi, Kohli trained at the West Delhi Cricket Academy and started his youth career with the Delhi Under-15 team. He made his international debut in 2008 and quickly became a key player in the ODI team and later made his Test debut in 2011. In 2013, Kohli reached the number one spot in the ICC rankings for ODI batsmen for the first time. During 2014 T20 World Cup, he set a record for the most runs scored in the tournament. In 2018, he achieved yet another milestone, becoming the world's top-ranked Test batsman, making him the only Indian cricketer to hold the number one spot in all three formats of the game. His form continued in 2019, when he became the first player to score 20,000 international runs in a single decade. In 2021, Kohli made the decision to step down as the captain of the Indian national team for T20Is, following the T20 World Cup and in early 2022 he stepped down as the captain of the Test team as well"

In [136]:
print(data)

Born and raised in New Delhi, Kohli trained at the West Delhi Cricket Academy and started his youth career with the Delhi Under-15 team. He made his international debut in 2008 and quickly became a key player in the ODI team and later made his Test debut in 2011. In 2013, Kohli reached the number one spot in the ICC rankings for ODI batsmen for the first time. During 2014 T20 World Cup, he set a record for the most runs scored in the tournament. In 2018, he achieved yet another milestone, becoming the world's top-ranked Test batsman, making him the only Indian cricketer to hold the number one spot in all three formats of the game. His form continued in 2019, when he became the first player to score 20,000 international runs in a single decade. In 2021, Kohli made the decision to step down as the captain of the Indian national team for T20Is, following the T20 World Cup and in early 2022 he stepped down as the captain of the Test team as well


In [138]:
l=data.split()

In [139]:
print(l)

['Born', 'and', 'raised', 'in', 'New', 'Delhi,', 'Kohli', 'trained', 'at', 'the', 'West', 'Delhi', 'Cricket', 'Academy', 'and', 'started', 'his', 'youth', 'career', 'with', 'the', 'Delhi', 'Under-15', 'team.', 'He', 'made', 'his', 'international', 'debut', 'in', '2008', 'and', 'quickly', 'became', 'a', 'key', 'player', 'in', 'the', 'ODI', 'team', 'and', 'later', 'made', 'his', 'Test', 'debut', 'in', '2011.', 'In', '2013,', 'Kohli', 'reached', 'the', 'number', 'one', 'spot', 'in', 'the', 'ICC', 'rankings', 'for', 'ODI', 'batsmen', 'for', 'the', 'first', 'time.', 'During', '2014', 'T20', 'World', 'Cup,', 'he', 'set', 'a', 'record', 'for', 'the', 'most', 'runs', 'scored', 'in', 'the', 'tournament.', 'In', '2018,', 'he', 'achieved', 'yet', 'another', 'milestone,', 'becoming', 'the', "world's", 'top-ranked', 'Test', 'batsman,', 'making', 'him', 'the', 'only', 'Indian', 'cricketer', 'to', 'hold', 'the', 'number', 'one', 'spot', 'in', 'all', 'three', 'formats', 'of', 'the', 'game.', 'His', 'f

In [140]:
len(l)

177

# Tokenizing Text Data

In [141]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
encoded_data= tokenizer.texts_to_sequences([data])[0]
encoded_data

[32,
 3,
 33,
 2,
 34,
 8,
 9,
 35,
 36,
 1,
 37,
 8,
 38,
 39,
 3,
 40,
 5,
 41,
 42,
 43,
 1,
 8,
 44,
 45,
 6,
 4,
 10,
 5,
 16,
 17,
 2,
 46,
 3,
 47,
 18,
 11,
 48,
 19,
 2,
 1,
 20,
 6,
 3,
 49,
 10,
 5,
 12,
 17,
 2,
 50,
 2,
 51,
 9,
 52,
 1,
 21,
 22,
 23,
 2,
 1,
 53,
 54,
 7,
 20,
 55,
 7,
 1,
 24,
 56,
 57,
 58,
 25,
 26,
 27,
 4,
 59,
 11,
 60,
 7,
 1,
 61,
 28,
 62,
 2,
 1,
 63,
 2,
 64,
 4,
 65,
 66,
 67,
 68,
 69,
 1,
 70,
 71,
 72,
 12,
 73,
 74,
 75,
 1,
 76,
 29,
 77,
 13,
 78,
 1,
 21,
 22,
 23,
 2,
 79,
 80,
 81,
 14,
 1,
 82,
 5,
 83,
 84,
 2,
 85,
 86,
 4,
 18,
 1,
 24,
 19,
 13,
 87,
 88,
 89,
 16,
 28,
 2,
 11,
 90,
 91,
 2,
 92,
 9,
 10,
 1,
 93,
 13,
 94,
 30,
 15,
 1,
 31,
 14,
 1,
 29,
 95,
 6,
 7,
 96,
 97,
 1,
 25,
 26,
 27,
 3,
 2,
 98,
 99,
 4,
 100,
 30,
 15,
 1,
 31,
 14,
 1,
 12,
 6,
 15,
 101]

# Vocabulary Size

In [142]:
vocab_size = len(tokenizer.word_index) + 1  # 0 is reserved for padding so that's why we added 1
print('Vocabulary Size: %d' % vocab_size)

Vocabulary Size: 102


# Creating Training Sequences

In [144]:
sequences = list()
for i in range(3, len(encoded_data)):
	sequence = encoded_data[i-3:i+1]
	sequences.append(sequence)
print('Total Sequences: %d' % len(sequences))

Total Sequences: 177


In [145]:
sequences = np.array(sequences)
X, y = sequences[:,0:3],sequences[:,-1]

In [146]:
sequences

array([[ 32,   3,  33,   2],
       [  3,  33,   2,  34],
       [ 33,   2,  34,   8],
       [  2,  34,   8,   9],
       [ 34,   8,   9,  35],
       [  8,   9,  35,  36],
       [  9,  35,  36,   1],
       [ 35,  36,   1,  37],
       [ 36,   1,  37,   8],
       [  1,  37,   8,  38],
       [ 37,   8,  38,  39],
       [  8,  38,  39,   3],
       [ 38,  39,   3,  40],
       [ 39,   3,  40,   5],
       [  3,  40,   5,  41],
       [ 40,   5,  41,  42],
       [  5,  41,  42,  43],
       [ 41,  42,  43,   1],
       [ 42,  43,   1,   8],
       [ 43,   1,   8,  44],
       [  1,   8,  44,  45],
       [  8,  44,  45,   6],
       [ 44,  45,   6,   4],
       [ 45,   6,   4,  10],
       [  6,   4,  10,   5],
       [  4,  10,   5,  16],
       [ 10,   5,  16,  17],
       [  5,  16,  17,   2],
       [ 16,  17,   2,  46],
       [ 17,   2,  46,   3],
       [  2,  46,   3,  47],
       [ 46,   3,  47,  18],
       [  3,  47,  18,  11],
       [ 47,  18,  11,  48],
       [ 18,  

# X Represents the Last Three words

In [147]:
X

array([[ 32,   3,  33],
       [  3,  33,   2],
       [ 33,   2,  34],
       [  2,  34,   8],
       [ 34,   8,   9],
       [  8,   9,  35],
       [  9,  35,  36],
       [ 35,  36,   1],
       [ 36,   1,  37],
       [  1,  37,   8],
       [ 37,   8,  38],
       [  8,  38,  39],
       [ 38,  39,   3],
       [ 39,   3,  40],
       [  3,  40,   5],
       [ 40,   5,  41],
       [  5,  41,  42],
       [ 41,  42,  43],
       [ 42,  43,   1],
       [ 43,   1,   8],
       [  1,   8,  44],
       [  8,  44,  45],
       [ 44,  45,   6],
       [ 45,   6,   4],
       [  6,   4,  10],
       [  4,  10,   5],
       [ 10,   5,  16],
       [  5,  16,  17],
       [ 16,  17,   2],
       [ 17,   2,  46],
       [  2,  46,   3],
       [ 46,   3,  47],
       [  3,  47,  18],
       [ 47,  18,  11],
       [ 18,  11,  48],
       [ 11,  48,  19],
       [ 48,  19,   2],
       [ 19,   2,   1],
       [  2,   1,  20],
       [  1,  20,   6],
       [ 20,   6,   3],
       [  6,   3

# Y Represents the Word to be Predicted

In [148]:
y

array([  2,  34,   8,   9,  35,  36,   1,  37,   8,  38,  39,   3,  40,
         5,  41,  42,  43,   1,   8,  44,  45,   6,   4,  10,   5,  16,
        17,   2,  46,   3,  47,  18,  11,  48,  19,   2,   1,  20,   6,
         3,  49,  10,   5,  12,  17,   2,  50,   2,  51,   9,  52,   1,
        21,  22,  23,   2,   1,  53,  54,   7,  20,  55,   7,   1,  24,
        56,  57,  58,  25,  26,  27,   4,  59,  11,  60,   7,   1,  61,
        28,  62,   2,   1,  63,   2,  64,   4,  65,  66,  67,  68,  69,
         1,  70,  71,  72,  12,  73,  74,  75,   1,  76,  29,  77,  13,
        78,   1,  21,  22,  23,   2,  79,  80,  81,  14,   1,  82,   5,
        83,  84,   2,  85,  86,   4,  18,   1,  24,  19,  13,  87,  88,
        89,  16,  28,   2,  11,  90,  91,   2,  92,   9,  10,   1,  93,
        13,  94,  30,  15,   1,  31,  14,   1,  29,  95,   6,   7,  96,
        97,   1,  25,  26,  27,   3,   2,  98,  99,   4, 100,  30,  15,
         1,  31,  14,   1,  12,   6,  15, 101])

In [149]:
y = to_categorical(y, num_classes=vocab_size)
y[:5]

array([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.

# Building Model 

In [150]:
model = Sequential()
model.add(Embedding(vocab_size, 30, input_length=3))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_12 (Embedding)    (None, 3, 30)             3060      
                                                                 
 lstm_16 (LSTM)              (None, 3, 50)             16200     
                                                                 
 lstm_17 (LSTM)              (None, 50)                20200     
                                                                 
 dense_12 (Dense)            (None, 102)               5202      
                                                                 
Total params: 44,662
Trainable params: 44,662
Non-trainable params: 0
_________________________________________________________________
None


In [151]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Training the Model

In [ ]:
model.fit(X, y, epochs=200)

Epoch 1/200
6/6 [==============================] - 2s 4ms/step - loss: 4.6243 - accuracy: 0.0904
Epoch 2/200
6/6 [==============================] - 0s 4ms/step - loss: 4.6188 - accuracy: 0.1073
Epoch 3/200
6/6 [==============================] - 0s 4ms/step - loss: 4.6133 - accuracy: 0.1073
Epoch 4/200
6/6 [==============================] - 0s 4ms/step - loss: 4.6059 - accuracy: 0.1073
Epoch 5/200
6/6 [==============================] - 0s 4ms/step - loss: 4.5963 - accuracy: 0.1073
Epoch 6/200
6/6 [==============================] - 0s 3ms/step - loss: 4.5816 - accuracy: 0.1073
Epoch 7/200
6/6 [==============================] - 0s 3ms/step - loss: 4.5607 - accuracy: 0.1073
Epoch 8/200
6/6 [==============================] - 0s 3ms/step - loss: 4.5222 - accuracy: 0.1073
Epoch 9/200
6/6 [==============================] - 0s 3ms/step - loss: 4.4618 - accuracy: 0.1073
Epoch 10/200
6/6 [==============================] - 0s 3ms/step - loss: 4.3617 - accuracy: 0.1073
Epoch 11/200
6/6 [===========